In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('data/shopify.csv', index_col=['created_at','order_id']).sort_index()
df.head()

,,shop_id,user_id,order_amount,total_items,payment_method
created_at,order_id,,,,,
2017-03-01 0:08:09,1863,39,738,536,4,cash
2017-03-01 0:10:19,1742,39,910,268,2,cash
2017-03-01 0:14:12,3229,97,912,324,2,cash
2017-03-01 0:19:31,1268,80,798,290,2,credit_card
2017-03-01 0:22:25,2690,49,799,258,2,credit_card


In [3]:
df.shape

(5000, 5)

In [4]:
df.describe()

# AOV = $3145.13
# max order_amount = 704000 could be problematic, bringing up our mean

,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.00000
mean,50.078800,849.092400,3145.128000,8.78720
std,29.006118,87.798982,41282.539349,116.32032
min,1.000000,607.000000,90.000000,1.00000
25%,24.000000,775.000000,163.000000,1.00000
50%,50.000000,849.000000,284.000000,2.00000
75%,75.000000,925.000000,390.000000,3.00000
max,100.000000,999.000000,704000.000000,2000.00000


In [5]:
# Ensure we have 100 unique sneaker shops
df['shop_id'].nunique()

100

In [6]:
# Ensure we have no null values, inspect Dtypes, and date range is 30 days as stated
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5000 entries, ('2017-03-01 0:08:09', 1863) to ('2017-03-30 9:55:00', 146)
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   shop_id         5000 non-null   int64 
 1   user_id         5000 non-null   int64 
 2   order_amount    5000 non-null   int64 
 3   total_items     5000 non-null   int64 
 4   payment_method  5000 non-null   object
dtypes: int64(4), object(1)
memory usage: 551.1+ KB


In [7]:
# Check order_amount = 704000
df[df['order_amount'] == 704000]

shop_id  user_id  order_amount  total_items  \
created_at         order_id                                                
2017-03-02 4:00:00 521            42      607        704000         2000   
                   4647           42      607        704000         2000   
2017-03-04 4:00:00 61             42      607        704000         2000   
2017-03-07 4:00:00 16             42      607        704000         2000   
                   2298           42      607        704000         2000   
2017-03-11 4:00:00 1437           42      607        704000         2000   
2017-03-12 4:00:00 2154           42      607        704000         2000   
2017-03-15 4:00:00 1363           42      607        704000         2000   
2017-03-17 4:00:00 1603           42      607        704000         2000   
2017-03-19 4:00:00 1563           42      607        704000         2000   
2017-03-22 4:00:00 4869           42      607        704000         2000   
2017-03-24 4:00:00 1105           42      607        704000         2000   
                   3333           42      607        704000         2000   
2017-03-25 4:00:00 4883           42      607        704000         2000   
2017-03-28 4:00:00 2836           42      607        704000         2000   
                   2970           42      607        704000         2000   
                   4057           42      607        704000         2000   

                            payment_method  
created_at         order_id                 
2017-03-02 4:00:00 521         credit_card  
                   4647        credit_card  
2017-03-04 4:00:00 61          credit_card  
2017-03-07 4:00:00 16          credit_card  
                   2298        credit_card  
2017-03-11 4:00:00 1437        credit_card  
2017-03-12 4:00:00 2154        credit_card  
2017-03-15 4:00:00 1363        credit_card  
2017-03-17 4:00:00 1603        credit_card  
2017-03-19 4:00:00 1563        credit_card  
2017-03-22 4:00:00 4869        credit_card  
2017-03-24 4:00:00 1105        credit_card  
                   3333        credit_card  
2017-03-25 4:00:00 4883        credit_card  
2017-03-28 4:00:00 2836        credit_card  
                   2970        credit_card  
                   4057        credit_card

• There are duplicate observations

• I will assume the duplicates that share the same `created_at` timestamp are erroneous

• I will assume that duplicates that have different `created_at` timestamps are correct

# Question 1

### a. There are duplicate entries with order_amount valued at $704,000. This is causing the mean to be higher than it should be. Based on this, I will assume there are more entries that are duplicated. 

While thinking about a better way to evaluate this data, one could either:

    1. Remove duplicates
    
    2. Find another metric

### 1. Remove duplicates

In [8]:
duplicates = df[df.duplicated()]
duplicates

shop_id  user_id  order_amount  total_items  \
created_at          order_id                                                
2017-03-01 23:27:41 489             9      790           118            1   
2017-03-02 4:00:00  4647           42      607        704000         2000   
2017-03-04 4:00:00  61             42      607        704000         2000   
2017-03-07 4:00:00  16             42      607        704000         2000   
                    2298           42      607        704000         2000   
...                               ...      ...           ...          ...   
2017-03-29 7:10:18  4899           91      726           160            1   
2017-03-29 9:26:09  3793           12      823           201            1   
2017-03-30 0:36:02  588            15      784           459            3   
2017-03-30 15:17:37 2516          100      899           111            1   
2017-03-30 16:35:46 1746           93      723           228            2   

                             payment_method  
created_at          order_id                 
2017-03-01 23:27:41 489               debit  
2017-03-02 4:00:00  4647        credit_card  
2017-03-04 4:00:00  61          credit_card  
2017-03-07 4:00:00  16          credit_card  
                    2298        credit_card  
...                                     ...  
2017-03-29 7:10:18  4899        credit_card  
2017-03-29 9:26:09  3793        credit_card  
2017-03-30 0:36:02  588         credit_card  
2017-03-30 15:17:37 2516              debit  
2017-03-30 16:35:46 1746              debit  

[67 rows x 5 columns]

In [9]:
df_drop_dup = df.drop(index=duplicates.index)

In [10]:
df_drop_dup.describe()

,shop_id,user_id,order_amount,total_items
count,4933.000000,4933.000000,4933.000000,4933.000000
mean,50.132577,849.979728,901.597811,2.401784
std,29.028649,86.887530,11347.962236,28.464316
min,1.000000,607.000000,90.000000,1.000000
25%,24.000000,776.000000,163.000000,1.000000
50%,51.000000,850.000000,284.000000,2.000000
75%,75.000000,925.000000,390.000000,3.000000
max,100.000000,999.000000,704000.000000,2000.000000


Average `order_amount` (AOV) is now 901.60, down from the original 3,145.13. This seems more realistic.

### 2. Find another metric

Order Value = price per shoe * total_items

AOV = sum of all Order Values / n # of orders

### b. Knowing that each store sells only one model of shoe, we know that despite our erroneous data set, the shoe price at every sneaker shop will remain constant. We know that all other metrics will be affected by duplicates in the data when calculating the average. Let's find the average shoe price per shop and call it `shoe_price`

In [11]:
# Feature engineer shoe price column
df_copy = df.copy()
df_copy['shoe_price'] = df_copy['order_amount'] / df_copy['total_items']
df_copy.head()

,,shop_id,user_id,order_amount,total_items,payment_method,shoe_price
created_at,order_id,,,,,,
2017-03-01 0:08:09,1863,39,738,536,4,cash,134.0
2017-03-01 0:10:19,1742,39,910,268,2,cash,134.0
2017-03-01 0:14:12,3229,97,912,324,2,cash,162.0
2017-03-01 0:19:31,1268,80,798,290,2,credit_card,145.0
2017-03-01 0:22:25,2690,49,799,258,2,credit_card,129.0


In [12]:
# Group by unique shops
df_prices = df_copy[['shop_id', 'shoe_price']].groupby(['shop_id']).mean()
df_prices

,shoe_price
shop_id,
1,158.0
2,94.0
3,148.0
4,128.0
5,142.0
...,...
96,153.0
97,162.0
98,133.0


In [13]:
df_prices.describe()

,shoe_price
count,100.000000
mean,407.990000
std,2557.462906
min,90.000000
25%,132.750000
50%,153.000000
75%,168.250000
max,25725.000000


### c. The average shoe price is 407.99.

This data needed to be cleaned up in order to find a more accurate Average Order Value (AOV). But assuming we are not able to clean the data, a better metric to evaluate the data is the Average Shoe Price. It took simple feature engineering by dividing `order_amount` by `total_items`, grouping these calculations by `shop_id`, then finding the average of this calculation across the entire dataset.

# Question 2:  SQL Questions

a. How many orders were shipped by Speedy Express in total?

SELECT ShipperName, COUNT(*)
FROM Orders o
JOIN Shippers s
ON o.ShipperID = s.ShipperID
WHERE ShipperName = 'Speedy Express'
GROUP BY ShipperName;

Answer: 54

b. What is the last name of the employee with the most orders?

SELECT LastName, COUNT(*) AS Count_Orders
FROM Orders o
JOIN Employees e
ON o.EmployeeID = e.EmployeeID
GROUP BY LastName
ORDER BY Count_Orders DESC
LIMIT 1;

Answer: 40

c. What product was ordered the most by customers in Germany?

WITH table1 AS (
SELECT * 
FROM Orders
WHERE CustomerID IN (SELECT CustomerID 
FROM Customers
WHERE Country = 'Germany')
)

SELECT ProductName, COUNT(ProductName) as Count_Orders
FROM table1 a
JOIN OrderDetails b
ON a.OrderID = b.OrderID
JOIN Products c
ON b.ProductID = c.ProductID
GROUP BY ProductName
ORDER BY Count_Orders DESC
LIMIT 1;

Answer: Gorgonzola Telino (5 orders)